In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support.ui import WebDriverWait

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# load the driver and URL

driver =webdriver.Chrome(r"D:\Softwares\chromedriver_win32\chromedriver.exe")
driver.maximize_window()
driver.implicitly_wait(5)
driver.get('https://www.amazon.in/')
time.sleep(4)

# now enter the details of product to be searched
product = input("Enter the product verticals to be searched : ")
search = driver.find_element_by_id("twotabsearchtextbox")
search.send_keys(product)
search_now = driver.find_element_by_id("nav-search-submit-button")
search_now.click()
time.sleep(5)

Enter the product verticals to be searched : guitars


### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
# for storing urls creating empty list and then adding urls of each product of first 3 pages
urls = []
for i in range(3):
    for j in driver.find_elements_by_xpath('//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a'):
        urls.append(j.get_attribute('href'))
    
    try:
        next_page=driver.find_element_by_xpath('//li[@class="a-last"]/a').get_attribute('href')
        driver.get(next_page)
        driver.refresh()
    except:
        pass
    
# creating empty lists to store data
brand_name= []
product_name = []
rating = []
no_of_rating = []
price = []
return_exchange =[]
expected_delivery = []
availability = []
other_details = []
product_url = [] 

for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    
    # scrape the brand name
    try:brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except:brand_name.append("-")
    # scrape the product name
    try:product_name.append(" ".join(driver.find_element_by_id("productTitle").text.split(' ')[1:]))
    except:product_name.append("-")
    # scrape the rating
    try:rating.append(driver.find_element_by_id("acrPopover").get_attribute('title').split(" ")[0])
    except:rating.append("-")
    # scrape the no. of ratings
    try:no_of_rating.append(driver.find_element_by_id("acrCustomerReviewText").text.split(" ")[0])
    except:no_of_rating.append("-")
    # scrape the price of product
    try:price.append(driver.find_element_by_id("priceblock_dealprice_lbl").text.split("₹ ")[1])
    except:
        try:price.append(driver.find_element_by_id("priceblock_ourprice").text.split("₹ ")[1])
        except:
            try:price.append(driver.find_element_by_id("priceblock_saleprice").text.split("₹ ")[1])
            except:price.append("-")
    # scrape product availability    
    try:
        a=driver.find_element_by_id("availability")
        availability.append(a.find_element_by_xpath('.//span').text)
    except:availability.append("-")
    # scrape expected delivery
    try:expected_delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]//b').text)
    except:expected_delivery.append("-")
    # scrape other details 
    try:other_details.append(driver.find_element_by_id("productDetails_techSpec_section_1").text.replace("\n"," , "))
    except:other_details.append("-") 
    # scrape whether return or exchange facilities
    try:return_exchange.append(driver.find_element_by_id("RETURNS_POLICY").find_element_by_xpath('//span[@class="a-declarative"]//div[2]//a').text)
    except:return_exchange.append("-")

# creating  datafrme
amazon = pd.DataFrame({})
amazon['Brand_Name']=brand_name
amazon['Name_Of_Product']=product_name
amazon['Rating_out_Of_5']=rating
amazon['No_Of_Ratings']=no_of_rating
amazon['Price']=price
amazon['Return/Exchange']=return_exchange
amazon['Expected_Delivery']=expected_delivery
amazon['Availability']=availability
amazon['Other_Details']=other_details
amazon['URL_of_Product']=product_url

# saving final scrapped datasetin csv format
amazon.to_csv('amazon_guitars.csv')
# view of scraped dataset
amazon

,Brand_Name,Name_Of_Product,Rating_out_Of_5,No_Of_Ratings,Price,Return/Exchange,Expected_Delivery,Availability,Other_Details,URL_of_Product
0,Kadence,"Frontier Series, Natural Acoustic Guitar With ...",3.8,64,"5,999.00",7 Days Replacement,"Monday, June 7",In stock.,"Brand Kadence , Manufacturer Kadence , Model K...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,"Frontier Series,Blue Acoustic Guitar With Die ...",4.0,236,"5,499.00",7 Days Replacement,"Monday, June 7",In stock.,Package Dimensions 96 x 47.4 x 12 cm; 3.26 Kil...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Kadence,Slowhand Premium Jumbo Size Semi Acoustic Guit...,4.4,125,"10,999.00",7 Days Replacement,"Monday, June 7",In stock.,"Brand Kadence , Manufacturer Kadence , Model S...",https://www.amazon.in/gp/slredirect/picassoRed...
3,JUAREZ,38 Inch Acoustic Guitar with Dual Action Truss...,3.0,20,"3,179.00",7 Days Replacement,"Saturday, June 5",In stock.,Product Dimensions 98 x 40 x 10 cm; 2 Kilogram...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Acoustic Guitar, 38 Inch Cutaway, 038C with Ba...",3.9,"9,560",-,7 Days Replacement,"Saturday, June 5",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.06 Kilog...,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
175,CBOX,4 String Guitar Children's Musical Instrument ...,5.0,1,849.00,7 Days Replacement,June 7 - 8,In stock.,"Brand CBOX , Manufacturer CBOX, CBOX , Model C...",https://www.amazon.in/CBOX-Childrens-Instrumen...
176,NICK'S,OPTIMIZE String Acoustic Guitar | Adjustable T...,4.0,2,879.00,7 Days Replacement,"Monday, June 7",In stock.,"Brand Nick's , Manufacturer Nick's , Model New...",https://www.amazon.in/gp/slredirect/picassoRed...
177,JUAREZ,"Arpeggio 39 Inch Acoustic Guitar Kit, Spruce T...",2.9,2,"4,935.00",7 Days Replacement,"Friday, June 4",In stock.,Product Dimensions 103 x 47 x 14 cm; 2.46 Kilo...,https://www.amazon.in/gp/slredirect/picassoRed...
178,JUAREZ,"Arpeggio 41 Inch Acoustic Guitar Kit, Spruce T...",5.0,1,"6,049.00",7 Days Replacement,"Saturday, June 5",In stock.,Product Dimensions 103 x 47 x 14 cm; 2.46 Kilo...,https://www.amazon.in/gp/slredirect/picassoRed...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [4]:
# creating function for scrapping images from google
def images(item):
    # load the driver and url
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get('https://images.google.com/?gws_rd=ssl')
    driver.maximize_window()
    driver.implicitly_wait(10)
    
    # entering the input in search bar and clicking the search button  
    search = driver.find_element_by_name("q").send_keys(item)
    time.sleep(3)
    driver.find_element_by_xpath('//span[@class="z1asCe MZy1Rb"]').click()
   

    print("start scrolling to generate more images on the page...")
    # we scroll down in order to generate more images on the website
    for _ in range(1000):
        driver.execute_script("window.scrollBy(0,10000)")
    
    # saving the path
    images=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

    # create empty lists
    image_url=[]
    image_data=[]
    # iterating
    for image in images:
        source = image.get_attribute('src')
        if source is not None:
            if(source[0:4] =='http'):
                image_url.append(source)
            
    # now specify the file location and then download the image 
    print("Downloading ...")
    for i in range(len(image_url)):
        if i>=100:
            break
        print("- {0} of {1} images" .format(i, 100),end="\n")
        response =requests.get(image_url[i])
        file =open(r"C:/Users/siwac/Desktop/"+item+"/image_"+str(i)+".jpg","wb")
        file.write(response.content)
    print("\n",20*" *"," Downloaded ",20*" *")

In [5]:
# using function finding the result
images('cars')
images('fruits')
images('machine learning')

start scrolling to generate more images on the page...
- 0 of 100 images
- 1 of 100 images
- 2 of 100 images
- 3 of 100 images
- 4 of 100 images
- 5 of 100 images
- 6 of 100 images
- 7 of 100 images
- 8 of 100 images
- 9 of 100 images
- 10 of 100 images
- 11 of 100 images
- 12 of 100 images
- 13 of 100 images
- 14 of 100 images
- 15 of 100 images
- 16 of 100 images
- 17 of 100 images
- 18 of 100 images
- 19 of 100 images
- 20 of 100 images
- 21 of 100 images
- 22 of 100 images
- 23 of 100 images
- 24 of 100 images
- 25 of 100 images
- 26 of 100 images
- 27 of 100 images
- 28 of 100 images
- 29 of 100 images
- 30 of 100 images
- 31 of 100 images
- 32 of 100 images
- 33 of 100 images
- 34 of 100 images
- 35 of 100 images
- 36 of 100 images
- 37 of 100 images
- 38 of 100 images
- 39 of 100 images
- 40 of 100 images
- 41 of 100 images
- 42 of 100 images
- 43 of 100 images
- 44 of 100 images
- 45 of 100 images
- 46 of 100 images
- 47 of 100 images
- 48 of 100 images
- 49 of 100 images
- 50 

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [6]:
# load the driver and url

driver = webdriver.Chrome(r"D:\Softwares\chromedriver_win32\chromedriver.exe")
driver.maximize_window()
driver.get('https://www.flipkart.com/')
driver.implicitly_wait(10)

In [7]:
# Before entering the output removing the login option
without_login=driver.find_element_by_xpath('//div[@class="_2QfC02"]/button')
without_login.click()

# finding the input option and clearing it and then sending the input we want
user_input = input("Enter product you want to search : ")
search=driver.find_element_by_class_name("_3704LK")
search.clear()
search.send_keys(str(user_input),Keys.ENTER)

# create empty list and then save all products urls
urls = []
for i in driver.find_elements_by_class_name("_1fQZEK"):
    urls.append(i.get_attribute("href"))

Enter product you want to search : smartphone


In [8]:
# Creating empty lists
brand_name =[]
smartphone_name =[]
color =[]
RAM =[]
storage =[]
primary_camera =[]
secondary_camera =[]
display =[]
resolution =[]
processor =[]
processor_core =[]
battery =[]
price =[]
product_url =[]

In [9]:
# now using for loop try to find the details of products
for url in urls:
    driver.get(url)
    product_url.append(url)
    driver.implicitly_wait(5)
    
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    
    # scrape brand name
    try:brand_name.append(driver.find_element_by_class_name("B_NuCI").text.split(' ')[0])
    except:brand_name.append("-")
    # scrape smartphone name
    try:smartphone_name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]/ul/li').text)
    except:smartphone_name.append("-")
    # scrape color of phone 
    try:color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]/ul/li').text)
    except:color.append("-")
    # scrape the Random Access Memory
    try:RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]/ul/li').text)
    except:RAM.append('-')
    # scrape storage of product      
    try:storage.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]/ul/li').text)
    except:storage.append('-')
    # scrape primary camera of product        
    try:primary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]/ul/li').text)
    except:primary_camera.append('-')
    # scrape secondary camera of product       
    try:secondary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]/ul/li').text)
    except:secondary_camera.append('-')
    # scrape the display   
    try:display.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]/ul/li').text.split(' (')[0])
    except:display.append('-')
    # scrape the resolution  
    try:resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]/ul/li').text.split(' Pixels')[0])
    except:resolution.append('-')
    # scrape the processor of product
    try:processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]/ul/li').text)
    except:processor.append('-')
    # scrape the core processor 
    try:processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]/ul/li').text)
    except:processor_core.append('-')
    # scrape the battery deatil of product
    try:
        driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr/td[1]').text == "Battery Capacity"
        battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr/td[2]/ul/li').text)
    except:
        try:
            driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[8]/table/tbody/tr/td[1]').text =="Battery Capacity"
            battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[8]/table/tbody/tr/td[2]/ul/li').text)
        except:battery.append("-")
    # scrape the price of product   
    try:price.append(driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]').text.split('₹')[1])
    except:price.append('-')

In [10]:
#create a DataFrame for our data
flipkart_phones=pd.DataFrame({})
flipkart_phones['Brand Name']=brand_name
flipkart_phones['Smartphone Name']=smartphone_name
flipkart_phones['Smartphone Colour']=color
flipkart_phones['RAM']=RAM
flipkart_phones['Storage (ROM)']=storage
flipkart_phones['Primray Camera']=primary_camera
flipkart_phones['Secondary Camera']=secondary_camera 
flipkart_phones['Display Size']=display
flipkart_phones['Display Resolution(Pixels)']=resolution
flipkart_phones['Processor']=processor
flipkart_phones['Processor Core']=processor_core
flipkart_phones['Battery']=battery
flipkart_phones['Price(₹)']=price
flipkart_phones['Product URL']=product_url

In [11]:
# view the scraped dataset
flipkart_phones

,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution(Pixels),Processor,Processor Core,Battery,Price(₹),Product URL
0,Infinix,Hot 10 Play,7° Purple,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm,1640 x 720,MediaTek Helio G35,Octa Core,6000 mAh,"8,499",https://www.flipkart.com/infinix-hot-10-play-7...
1,REDMI,9 Power,Mighty Black,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"9,999",https://www.flipkart.com/redmi-9-power-mighty-...
2,Infinix,Hot 10 Play,Morandi Green,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm,1640 x 720,MediaTek Helio G35,Octa Core,6000 mAh,"8,499",https://www.flipkart.com/infinix-hot-10-play-m...
3,realme,Narzo 30A,Laser Blue,3 GB,32 GB,13MP + 2MP,Yes,16.54 cm,1600 x 720,MediaTek Helio G85,Octa Core,6000 mAh,"8,499",https://www.flipkart.com/realme-narzo-30a-lase...
4,REDMI,9 Power,Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"9,999",https://www.flipkart.com/redmi-9-power-electri...
5,realme,Narzo 30A,Laser Black,3 GB,32 GB,13MP + 2MP,Yes,16.54 cm,1600 x 720,MediaTek Helio G85,Octa Core,6000 mAh,"8,499",https://www.flipkart.com/realme-narzo-30a-lase...
6,realme,Narzo 30A,Laser Black,4 GB,64 GB,13MP + 2MP,Yes,16.54 cm,1600 x 720,MediaTek Helio G85,Octa Core,6000 mAh,"9,499",https://www.flipkart.com/realme-narzo-30a-lase...
7,realme,Narzo 30A,Laser Blue,4 GB,64 GB,13MP + 2MP,Yes,16.54 cm,1600 x 720,MediaTek Helio G85,Octa Core,6000 mAh,"9,499",https://www.flipkart.com/realme-narzo-30a-lase...
8,realme,Narzo 20,Glory Silver,4 GB,64 GB,48MP + 8MP + 2MP,8MP Front Camera,16.56 cm,1600 x 720,MediaTek Helio G85,Octa Core,6000 mAh,"10,499",https://www.flipkart.com/realme-narzo-20-glory...
9,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm,2340 x 1080,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"10,999",https://www.flipkart.com/poco-m3-cool-blue-64-...


### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [12]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
driver.get('https://www.google.co.in/maps')
driver.implicitly_wait(5)
time.sleep(3)

def google_map(city):
    
    search = driver.find_element_by_id("searchboxinput")
    search.send_keys(city)
    time.sleep(3)
    search.send_keys(Keys.ENTER)
    # now scrapping latitude and longitude from url
    latitude = driver.current_url.split("@")[1].split(',')[0]
    longitude = driver.current_url.split("@")[1].split(',')[1]
    print(city +"\nLatitude is -> "+latitude+"and Longitude -> "+longitude)

In [13]:
google_map('Chandigarh')

Chandigarh
Latitude is -> 29.3235154and Longitude -> 76.3198228


### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [14]:
# creating a function for extracting the data of funding deals
def fund_track(url):
    
    #load the driver  and url
    driver = webdriver.Chrome("chromedriver.exe")
    driver.maximize_window()
    driver.get(url)
    driver.implicitly_wait(10)

    # selecting the fundings options and going to funding page
    funding_deals = driver.find_element_by_xpath('//*[@id="menu-item-51510"]/a').get_attribute('href')
    driver.get(funding_deals)
    
    # creating empty lists
    date =[]
    startup_name =[]
    industrial_vertical =[]
    sub_vertical =[]
    location =[]
    investor =[]
    investment =[]
    amount =[]
    
    # setting range according to selection of data we want to scrap
    for i in range(48,51):
        driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[2]'.format(i)).click()
       
        # scrape the date
        for a in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i)):
            date.append(a.text)
    
        # scrape the startup name
        for b in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]/a'.format(i)):
            startup_name.append(b.text)
      
        # scrape industrial vertical
        for c in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i)):
            industrial_vertical.append(c.text)
     
        # scrape sub vertical
        for d in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i)):
            sub_vertical.append(d.text)
     
        # scrape the location
        for e in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i)):
            location.append(e.text)
     
        # scrape the investor name
        for f in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i)):
            investor.append(f.text)
     
        # scrape the investment
        for g in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i)):
            investment.append(g.text)
     
        # srape the amount
        for h in driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i)):
            amount.append(h.text)
    # creating dataframe        
    import pandas as pd 
    funding_deals = pd.DataFrame({})
    funding_deals['Date(dd/mm/yyyy)']=date
    funding_deals['Startup Name']=startup_name
    funding_deals['Industry / Vertical']=industrial_vertical
    funding_deals['Sub-Vertical']=sub_vertical
    funding_deals['City / Location']=location
    funding_deals["Investor's Name"]=investor
    funding_deals['Investment Type']=investment
    funding_deals['Amount(in USD)']=amount
    
    return funding_deals

In [15]:
# using function for getting the scraped dataset
fund_track('https://trak.in/')

,Date(dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investor's Name,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


### 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [16]:
# load the driver 
driver = webdriver.Chrome("chromedriver.exe")

# maximising the window
driver.maximize_window()

# load the url
driver.get('https://www.digit.in/')
driver.implicitly_wait(10)

# using xpath for going to the required page of top 10 gaming laptops
driver.find_element_by_xpath('//div[@class="menu"]/ul/li[4]/a').click()
driver.find_element_by_xpath('//div[@class="listing_container"]//ul/li[26]/a').click()

In [17]:
# create empty lists
lap_name =[]
lap_windows =[]
lap_display =[]
lap_processor =[]
lap_memory =[]
lap_weight =[]
lap_dimension =[]
lap_graphics =[]
lap_price =[]
lap_release_date =[]
lap_official_website =[]

# scrape laptop name
for k in driver.find_elements_by_xpath('//div[@data-cat="Top_Ten_En_Scroll"]/a/h3'):
    lap_name.append(k.text)
    
# scrape windows present in laptop    
WINDOWS=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
windows =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(WINDOWS)):
    if WINDOWS[i].text=="OS":
        lap_windows.append(windows[i].text)

# scrape display of laptop
DISPLAY=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
display =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(DISPLAY)):
    if DISPLAY[i].text=="Display":
        lap_display.append(display[i].text)

# scrape processor of laptop
PROCESSOR=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
processor =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(PROCESSOR)):
    if PROCESSOR[i].text=="Processor":
        lap_processor.append(processor[i].text)

# scrape memory of laptop
MEMORY=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
memory =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(MEMORY)):
    if MEMORY[i].text=="Memory":
        lap_memory.append(memory[i].text)
        
# scrape weight of laptop        
WEIGHT=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
weight =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(WEIGHT)):
    if WEIGHT[i].text=="Weight":
        lap_weight.append(weight[i].text)

# scrape dimension of laptop
DIMENSION=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
Dimension =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(DIMENSION)):
    if DIMENSION[i].text=="Dimension":
        lap_dimension.append(Dimension[i].text)

# scrape graphics of laptop
GRAPHICS=driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[1]')
graphics =driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr/td[3]')
for i in range(len(GRAPHICS)):
    if GRAPHICS[i].text=="Graphics Processor":
        lap_graphics.append(graphics[i].text)

# now getting each laptop url and then find other details
urls =[]
for i in driver.find_elements_by_xpath('//span[@class="datahreflink spec"]'):
    urls.append(i.get_attribute('data-href'))
    
for a in urls:
    driver.get(a)
    # scrape laptop price
    try:lap_price.append(driver.find_element_by_xpath('//div[@class="Block-price"]/b').text)
    except:lap_price.append('-')
    # scrape laptop release date    
    try:lap_release_date.append(driver.find_element_by_xpath('//div[@class="status-bar only-desktop"]/div[2]/b').text)
    except:lap_release_date.append('-')
    # scrape laptop official website   
    try:lap_official_website.append(driver.find_element_by_xpath('//div[@class="status-bar only-desktop"]/div[3]/b').text)
    except:lap_official_website.append('-')
        
# now creating dataframe of scraped data        
import pandas as pd
top_10_gaming_laptops=pd.DataFrame({})
 
top_10_gaming_laptops['Laptop Model']=lap_name
top_10_gaming_laptops['Operating System']=lap_windows
top_10_gaming_laptops['Display']=lap_display
top_10_gaming_laptops['Processor']=lap_processor
top_10_gaming_laptops['Memory']=lap_memory
top_10_gaming_laptops['Weight(kg)']=lap_weight
top_10_gaming_laptops['Dimension']=lap_dimension
top_10_gaming_laptops['Graphics']=lap_graphics
top_10_gaming_laptops['Release Date']=lap_release_date
top_10_gaming_laptops['Official Website']=lap_official_website
top_10_gaming_laptops['Price(₹)']=lap_price

In [18]:
# viewing the scraped dataset
top_10_gaming_laptops

,Laptop Model,Operating System,Display,Processor,Memory,Weight(kg),Dimension,Graphics,Release Date,Official Website,Price(₹)
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"28 Apr, 2021",Alienware,-
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"28 Apr, 2021",Alienware,"341,990"
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"28 Apr, 2021",Asus,-
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"12 Aug, 2020",Asus,"164,990"
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"28 Apr, 2021",Lenovo,"71,990"
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"20 Nov, 2020",Asus,"199,990"
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"09 Apr, 2021",Acer,"56,990"
